In [ ]:
import pandas as pd
from pyprojroot import here
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from nutils import interval_score, colordict, name_mask

In [ ]:
TARGET = 'occ'
FS = 'a'
T = 80
HPO = 1

In [ ]:
true_matrix = pd.read_csv(
    here() / f'data/processed/true_matrices/{TARGET}.csv',
    index_col='datetime',
    parse_dates=True
)

In [ ]:
result_list = list()

data_path = Path('data/processed/prediction_matrices/')

for p in here(data_path / '50').glob(f'*-1.csv'):
    result = dict()
    
    parts = p.stem.split('-')
    model = parts[1]
    fs = parts[2]
    
    if fs=='f':
        fs='a'
    
    if fs==FS:
        pred_matrix = pd.read_csv(
            p, 
            index_col='datetime', 
            parse_dates=True)

        idx = pred_matrix.dropna().index.intersection(true_matrix.dropna().index)

        pred_vector = pred_matrix.loc[idx]
        true_vector = true_matrix.loc[idx]

        # mae
        error_vector = np.abs(pred_vector - true_vector).mean()
        error_vector.name = model
        result_list.append(error_vector)

In [ ]:
errors = pd.concat(result_list, axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

errors.plot(ax=ax,
            color=[colordict.get(x, 'grey') for x in errors.columns])

ax.set_ylim(bottom=0, top=16)
ax.set_xlim(left=0, right=23)
ax.legend([name_mask[x] for x in errors.columns], frameon=False, loc=4);
ax.set_ylabel('MAE');
ax.set_xlabel('Forecast horizon / Hour of day');
ax.grid(lw=.3, ls='--')

plt.savefig(here() / f'output/plots/horizon_mae-{FS}-{HPO}.jpg', 
            dpi=300, bbox_inches='tight')

In [ ]:
diff = np.abs(errors - errors.iloc[0]) / errors.iloc[0]
diff = (diff * 100).round(0).astype(int)
diff.to_latex(
    buf=here() / f'output/tables/horizon_mae-{FS}.tex',
)